In [0]:
# install chromium, its driver, and selenium
!apt-get update ###
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
!pip install selenium

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

wd = webdriver.Chrome('chromedriver', options=options)

In [0]:
url = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%EC%9D%B4%EB%8C%80%EC%97%AD+%EC%B9%B4%ED%8E%98'
wd.get(url)
print(wd.page_source)

In [0]:
from bs4 import BeautifulSoup

In [0]:
soup = BeautifulSoup(wd.page_source, "html.parser")
a = soup.select("#main_pack > div.blog.section._blogBase._prs_blg > div > span")
print(a[0].text)

1-10 / 47,992건


In [0]:
### 검색어 입력
wd.execute_script("document.getElementById('nx_query').value='이대역 카페'")
wd.implicitly_wait(3)
# 검색 버튼 클릭
e = wd.find_element_by_css_selector('#nx_search_form > fieldset > button')
wd.execute_script("arguments[0].click()", e)
wd.implicitly_wait(3)

### 기간 입력
# '기간' 클릭
e1 = wd.find_element_by_css_selector('#snb > div > ul > li:nth-of-type(2) > a')
wd.execute_script("arguments[0].click();", e1)
wd.implicitly_wait(3)
# 시작날짜 입력
#e2 = wd.find_element_by_css_selector('#blog_input_period_begin')
wd.execute_script("document.getElementById('blog_input_period_begin').value='2014.12.01'")
wd.implicitly_wait(3)
# 끝날짜 입력
#e3 = wd.find_element_by_css_selector('#blog_input_period_end')
wd.execute_script("document.getElementById('blog_input_period_end').value='2014.12.31'")
wd.implicitly_wait(3)
# '적용하기' 클릭
e2 = wd.find_element_by_css_selector('#_nx_option_date > div > span > button')
wd.execute_script("arguments[0].click()", e2)
wd.implicitly_wait(3)

### 건수 출력
soup = BeautifulSoup(wd.page_source, "html.parser")
a = soup.select("#main_pack > div.blog.section._blogBase._prs_blg > div > span")
print(a[0].text)

1-10 / 103건


### 월별 / 지하철역별

In [0]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [27]:
date = []
date_begin = datetime.date(2019, 1, 1)
for i in range(4): date.append(date_begin + relativedelta(months=i))
print(date)

[datetime.date(2019, 1, 1), datetime.date(2019, 2, 1), datetime.date(2019, 3, 1), datetime.date(2019, 4, 1)]


In [15]:
from google.colab import drive 
drive.mount('/content/gdrive')

subway = pd.read_csv('gdrive/My Drive/subway.csv', encoding='utf8', header=None)
subway

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,0
0,독바위
1,문래
2,선정릉
3,한성백제
4,사평
5,삼양
6,독산
7,혜화
8,명일
9,흑석


In [0]:
blog = pd.DataFrame('', columns=[format(d) for d in date], index=subway[0])

In [0]:
for s in subway[0]:

  #키워드 검색
  keyword = s + '역 카페'
  wd.execute_script("""var keyword = arguments[0];
    document.getElementById('nx_query').value = keyword;""", keyword)
  e1 = wd.find_element_by_css_selector('#nx_search_form > fieldset > button')
  wd.execute_script("arguments[0].click();", e1)
  wd.implicitly_wait(3)
 
  #기간 설정
  e2 = wd.find_element_by_css_selector('#snb > div > ul > li:nth-of-type(2) > a')
  wd.execute_script("arguments[0].click();", e2)
  wd.implicitly_wait(3)

  for d in date:
    d_begin = ".".join(format(d).split('-'))
    d_end = ".".join(format(d + relativedelta(months=1) - relativedelta(days=1)).split('-'))
    
    wd.execute_script("""var d_begin = arguments[0];
      document.getElementById('blog_input_period_begin').value = d_begin;""", d_begin)
    wd.execute_script("""var d_end = arguments[0];
      document.getElementById('blog_input_period_end').value = d_end;""", d_end)
    e3 = wd.find_element_by_css_selector('#_nx_option_date > div > span > button')
    wd.execute_script("arguments[0].click();", e3)
    wd.implicitly_wait(3)

    #블로그 건수 저장
    soup = BeautifulSoup(wd.page_source, "html.parser")
    n = soup.select("#main_pack > div.blog.section._blogBase._prs_blg > div > span")
    if n:
      n_blog = n[0].text.split("/")[1].lstrip().replace(",", "").replace("건", "")
      blog.loc[s][format(d)] = n_blog

In [30]:
blog

,2019-01-01,2019-02-01,2019-03-01,2019-04-01
0,,,,
독바위,20,16,25,23
문래,173,167,207,176
선정릉,185,167,221,425
한성백제,44,36,39,49
사평,35,26,30,24
삼양,2,2,7,7
독산,30,42,62,94
혜화,778,659,667,628
명일,55,44,67,66


In [0]:
blog.to_csv("gdrive/My Drive/blog.csv", encoding="utf8")